In [70]:
import pickle
import os
import xlsxwriter
import pandas as pd
import numpy as np

def loadData(pickle_name):
    # for reading also binary mode is important
    dbfile = open(pickle_name, 'rb')    
    db = pickle.load(dbfile)
    dbfile.close()
    return db
    
# print(loadData("../re.pickle"))

db = loadData("../re.pickle")

def create_sum(data):
    grouped_dict = {}
    for x in db:
        if x['batch_size'] not in grouped_dict:
            grouped_dict[x['batch_size']] = [x]
        else:
            grouped_dict[x['batch_size']].append(x)
    
    return grouped_dict

def set_style(df, sheet_name, no=0):
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    

    border_fmt = workbook.add_format({'bottom':1, 'top':1, 'left':1, 'right':1})

    worksheet.conditional_format(xlsxwriter.utility.xl_range(0, 0, len(df), len(df.columns) + no), {'type': 'no_errors', 'format': border_fmt})
    #worksheet.conditional_format(xlsxwriter.utility.xl_range(0, 0, len(df), len(df.columns)+1), {'type': 'no_errors', 'format': fmt})
    
    
def save(df_data):
    """
    Save single dataframe
    Args:
        df_data (_type_): the require format of dataframe

    Returns:
        _type_: return dataframe instance
    """
    multi_index = pd.MultiIndex.from_tuples([("4", "2.8Ghz"), ("8", "3.0Ghz"), ("12", "3.2Ghz"), ("16", "3.4Ghz"), ("20", "3.6Ghz"),
                                             ("24", "3.8Ghz")], names=['core', 'fre'])
                                       
             
    cols = pd.MultiIndex.from_tuples([("Avg"), ("1st"), ("2nd")])
                                    
    df = pd.DataFrame(df_data, columns=cols, index=multi_index)
    
    return df 

def set_index(core):
    multi_index = pd.MultiIndex.from_tuples([(core, "2.8Ghz"), (core, "3.0Ghz"), (core, "3.2Ghz"), (core, "3.4Ghz"), (core, "3.6Ghz"),
                                             (core, "3.8Ghz")], names=['core', 'fre'])
    
    return multi_index

with pd.ExcelWriter('output-2.xlsx', engine='xlsxwriter') as writer:
    cols = ["Avg", "1st", "2nd"] 
    bs_class = create_sum(db)
    for key, value in bs_class.items():

        for bs_sample in value:
            sheet_name = '{0}_{1}'.format(bs_sample['precision'], int(key))
            sun = list(bs_sample['kpi'].keys())
        index = [i for i in sun]
        
        multi_index = pd.MultiIndex.from_tuples([("50", "2.8Ghz"), ("50", "3.0Ghz"), ("50", "3.2Ghz"), ("50", "3.4Ghz"), ("50", "3.6Ghz"),
                                             ("50", "3.8Ghz")], names=['core', 'fre'])
        
        df = [ pd.DataFrame(list(bs_value['kpi'].values()), columns=cols,index=set_index(bs_value['core'])).reset_index('fre') for bs_value in value ]


        em_multi_index = pd.MultiIndex.from_tuples([(np.nan, np.nan)], names=['core', 'fre'])
        df_and_empty = []
        for d in df:
            df_and_empty.append(d)
            empty_df = pd.DataFrame([[np.nan] * len(d.columns)], columns=d.columns, index=em_multi_index).reset_index('fre', drop=True)
            df_and_empty.append(empty_df)
    
        x= pd.concat(df_and_empty)
        x.to_excel(writer, sheet_name=sheet_name)
        set_style(x, sheet_name)


In [60]:
# Create MultiIndex pandas DataFrame (Multi level Index)
import pandas as pd
index = [ x for x in range(6)]
print(index)
multi_index = pd.MultiIndex.from_tuples([("50", "2.8Ghz"), ("50", "3.0Ghz"), ("50", "3.2Ghz"), ("50", "3.4Ghz"), ("50", "3.6Ghz"),
                                             ("50", "3.8Ghz")], names=['core', 'fre'])

cols = ["Avg", "1st", "2nd"] 

data=[[100,300, 900 ], [200,500, 300], [200,500, 300], [200,500, 300], [200,500, 300], [200,500, 300]]

df = pd.DataFrame([[np.nan] * len(d.columns)], columns=d.columns, index=multi_index)


# df = pd.DataFrame(data, columns=cols,index=multi_index)

print(df)
df2= df.reset_index('fre', drop=True)
print(df2)
# print(df2.to_excel("output.xlsx"))


[0, 1, 2, 3, 4, 5]
             fre  Avg  1st  2nd
core fre                       
50   2.8Ghz  NaN  NaN  NaN  NaN
     3.0Ghz  NaN  NaN  NaN  NaN
     3.2Ghz  NaN  NaN  NaN  NaN
     3.4Ghz  NaN  NaN  NaN  NaN
     3.6Ghz  NaN  NaN  NaN  NaN
     3.8Ghz  NaN  NaN  NaN  NaN
      fre  Avg  1st  2nd
core                    
50    NaN  NaN  NaN  NaN
50    NaN  NaN  NaN  NaN
50    NaN  NaN  NaN  NaN
50    NaN  NaN  NaN  NaN
50    NaN  NaN  NaN  NaN
50    NaN  NaN  NaN  NaN
